In [1]:
import os
from pathlib import Path
from typing import Final
import tensorflow as tf
import numpy as np
import cv2
import utils

In [2]:
EPOCHS: Final[int] = 4
ACCURACY: Final[int] = 97.53
model = tf.keras.models.load_model(os.path.join(
    utils.DATA_DIR, 
    "models", 
    str(EPOCHS), 
    f"mnist-{EPOCHS}-{ACCURACY}.model"
))

In [3]:
test_images_paths: list[Path] = [
    os.path.join(utils.TEST_DIR, f) 
    for f in os.listdir(utils.TEST_DIR) 
    if f.endswith(".png")
]

In [4]:
utils.log_to_file(f"Starting testing model with EPOCHS: {EPOCHS} and ACCURACY: {ACCURACY}")

num_faults: int = 0
for image_path in test_images_paths:
    image: np.ndarray = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image_array: np.ndarray = np.invert(np.array([image]))
    predected: np.ndarray = model.predict(image_array)
    image_name: str = f"Predicted: {np.argmax(predected)}"
    # show_image(image, image_name)
    utils.log_to_file(f"predected: {np.argmax(predected)} and it was {image_path.split('/')[-1].split('.')[0]}")
    if np.argmax(predected) != int(image_path.split('/')[-1].split('.')[0]):
        num_faults += 1

utils.log_to_file(f"Accuracy: {100 - (num_faults / len(test_images_paths)) * 100}%")

1/1 [==============================] - 0s 9ms/step
